# Project Basics Of Mobile Robotics

### Computer Vision


### Global navigation

A common pathfinding algorithm in computer science and artificial intelligence is the A* (A-star) algorithm. Its purpose is to determine the shortest path on a graph or grid between a starting point and a goal point. Combining the ideas of Greedy Best-First Search and Dijkstra's algorithm, A* uses a heuristic function to effectively direct the search.

Important attributes:

Optimality: If certain requirements are satisfied, A* ensures that the shortest path will be found. 

Heuristic Guidance: A* estimates the cost from the current node to the goal using a heuristic function, commonly referred to as the "h-value". By directing the search, this heuristic increases algorithmic efficiency by examining the most promising paths first.

Various methods can be employed to apply the A* algorithm on a map, including the fixed-grid size approach, Voronoi diagrams, visibility graphs, and cell decomposition methods. Each method offers distinct advantages and considerations for pathfinding applications.

In our project, we assessed two primary pathfinding approaches: the fixed-grid size method and the visibility graph. Although the visibility graph demonstrated a slightly faster computation time, the difference was not substantial. However, a critical drawback surfaced with the visibility graph method, as it significantly reduced the number of reachable points, resulting in suboptimal solutions.

You can see our visibility graph below: 

<img src="images\vis_graph.jpeg" alt="Visibility Graph" style="width: 500px;"/>


Given that computation time differences were marginal, we decided to favor the fixed-grid size approach. This method involves dividing the map into uniform squares of predetermined cell sizes and utilizing obstacle masks generated through computer vision to construct a grid. The A* algorithm is then applied to this grid to determine the most efficient path.

We are displaying the A* exploration in real time, and an screenshot of such display can be seen below:

<img src="images\Astar_explore.png" alt="Exploration" style="width: 500px;"/>

We are also displaying the grid and the found path on the the map. You can see the screenshot of them below:

<img src="images\Astar_path.png" alt="Path" style="width: 500px;"/>






In [ ]:
# imports
import heapq
import cv2
from computer_vision import * 
import numpy as np

# possible moves from one square to another
moves_8n = [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]

moves_4n = [(0, -1), (0, 1), (-1, 0), (1, 0)]



class Node:
    """
    Represents a node in the A* algorithm.

    Attributes:
    - parent (Node): The parent node in the search tree.
    - position (Tuple[int, int]): The position (coordinates) of the node on the grid.
    - cost_of_move (float): The cost of reaching this node from the start.
    - heuristic (float): The estimated cost to reach the goal from this node.
    - total_cost (float): The sum of the cost_of_move and heuristic.

    Methods:
    - __eq__(self, other): Compares two nodes based on their positions.
    - __lt__(self, other): Compares two nodes based on their total_cost 
    
    """
    def __init__(self, parent=None, position=None):

        self.parent = parent
        self.position = position

        self.cost_of_move = 0

        self.heuristic = 0
        self.total_cost = 0


    def __eq__(self, other):

        return self.position == other.position


    def __lt__(self, other):

        if self.total_cost == other.total_cost:

            return id(self) < id(other)

        return self.total_cost < other.total_cost


def astar_grid(maze, start, end, moves, map_copy):
    """
    A* search algorithm for finding the shortest path on a 2D grid.

    Parameters:
    - maze (numpy.ndarray): A 2D grid where 0 represents an open path, and 1 represents an obstacle.
    - start (Tuple[int, int]): The starting point on the grid.
    - end (Tuple[int, int]): The destination point on the grid.
    - moves (List[Tuple[int, int]]): A list of tuples representing possible moves from a given position.

    Returns:
    - np.array[Tuple[int, int]] or None: The shortest path from the start to the end on the grid, or None if no path is found.

    Node Class:
    - The algorithm uses a Node class to represent a point in the search space, which includes information about the node's position,
      cost of movement from the start, heuristic (estimated cost to reach the goal), and the total cost.

    Assumptions:
    - The maze is represented as a NumPy array where 0 denotes an open path, and 1 denotes an obstacle.

    Example:
    >>> maze = np.array([[0, 0, 0, 1, 0],
    ...                  [0, 1, 0, 1, 0],
    ...                  [0, 1, 0, 0, 0],
    ...                  [0, 0, 0, 1, 0],
    ...                  [0, 0, 0, 0, 0]])
    >>> start = (0, 0)
    >>> end = (4, 4)
    >>> moves_8n = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (-1, -1), (-1, 1), (1, -1)]
    >>> path = astar_grid(maze, start, end, moves_8n)
    """

    start_node = Node(None, start)

    start_node.cost_of_move = start_node.heuristic = start_node.total_cost = 0

    end_node = Node(None, end)

    end_node.cost_of_move = end_node.heuristic = end_node.total_cost = 0

    open_list = []
    closed_set = set()
    visited_positions = set()  # Maintain a set of visited positions

    heapq.heappush(open_list, start_node)
    visited_positions.add(start)

    while open_list:

        current_node = heapq.heappop(open_list)

        closed_set.add(current_node.position)

        cv2.circle(map_copy, (int((current_node.position[0] * map_copy.shape[1]) / len(maze[0])),
                              int((current_node.position[1] * map_copy.shape[0]) / len(maze))), 2, (125, 0, 55), -1)
        cv2.imshow('Map_copy', map_copy)
        cv2.waitKey(1)

        if current_node == end_node:
            path = []
            while current_node:
                path.append(current_node.position)

                current_node = current_node.parent

            return path[::-1]

        children = [
            Node(current_node, (current_node.position[0] + dx, current_node.position[1] + dy))

            for dx, dy in moves
            if (

                    0 <= current_node.position[0] + dx < len(maze[0])

                    and 0 <= current_node.position[1] + dy < len(maze)

                    and maze[current_node.position[1] + dy][current_node.position[0] + dx] == 0

                    and (current_node.position[0] + dx, current_node.position[1] + dy) not in closed_set
                    and (current_node.position[0] + dx, current_node.position[1] + dy) not in visited_positions
            )
        ]

        for child in children:

            dx, dy = child.position[0] - current_node.position[0], child.position[1] - current_node.position[1]

            child.cost_of_move = current_node.cost_of_move + 1.41 if dx != 0 and dy != 0 else current_node.cost_of_move + 1

            child.heuristic = abs(child.position[0] - end_node.position[0]) \
                              + abs(child.position[1] - end_node.position[1])

            child.total_cost = child.cost_of_move + child.heuristic

            heapq.heappush(open_list, child)
            visited_positions.add(child.position)



def simplify_path(path):

    """
    Simplifies a path by removing unnecessary intermediate points.

    Parameters:
    - path (np.array[Tuple[int, int]]): The original path represented as a list of coordinates.

    Returns:
    - np.array[Tuple[int, int]]: The simplified path with unnecessary intermediate points removed.
    
    Example:
    >>> simplify_path([(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2)])
    [(0, 0), (3, 0), (3, 2)]

    """
    simplified_path = [path[0]] 

    for i in range(1, len(path) - 1):
        current_point = np.array(path[i])
        next_point = np.array(path[i + 1])
        direction_vector = next_point - np.array(simplified_path[-1])
        
        if np.cross(direction_vector, current_point - np.array(simplified_path[-1])) == 0:
            continue  

        simplified_path.append(path[i]) 

    simplified_path.append(path[-1])

    return simplified_path

def make_path(map_img, obstacle_masks, cell_size, start, end, grid,  metric_padding, map_x = 600, map_y = 400):
    """
    Generates a path on a grid-based map using A* algorithm, considering obstacles on the image.

    Parameters:
    - map_img (numpy.ndarray): The original map image.
    - obstacle_masks (List[numpy.ndarray]): List of obstacle masks on the map.
    - cell_size (int): Size of each grid cell.
    - start (Tuple[int, int]): Starting point in image coordinates (x, y).
    - end (Tuple[int, int]): Ending point in image coordinates (x, y).
    - grid (numpy.ndarray): The grid representing the map.
    - map_x (int): Width of the map in image coordinates.
    - map_y (int): Height of the map in image coordinates.

    Returns:
    - Tuple[numpy.ndarray, List[Tuple[int, int]], List[Tuple[int, int]], List[Tuple[float, float]]]:
        - grid (numpy.ndarray): The grid with obstacle information.
        - path_grid (List[Tuple[int, int]]): The path on the grid coordinates.
        - simplified_path (List[Tuple[int, int]]): The simplified path on the grid coordinates.
        - metric_path (List[Tuple[float, float]]): The path in metric (image) coordinates.

    Example:
    >>> map_img = cv2.imread('map_image.png')
    >>> obstacle_masks = [obstacle_mask_1, obstacle_mask_2]
    >>> cell_size = 10
    >>> start = (100, 50)
    >>> end = (300, 200)
    >>> grid, path_grid, simplified_path, metric_path = make_path(map_img, obstacle_masks, cell_size, start, end, grid)
    """
    map_copy = map_img.copy()
    bw_map = cv2.cvtColor(map_img.copy(), cv2.COLOR_BGR2GRAY)
    grid = create_grid(bw_map, obstacle_masks, cell_size, metric_padding)
    start_grid = (grid.shape[1] * start[0] // map_img.shape[1], grid.shape[0] * start[1] // map_img.shape[0])
    end_grid = (grid.shape[1] * end[0] // map_img.shape[1], grid.shape[0] * end[1] // map_img.shape[0])
    path_grid = astar_grid(grid, start_grid, end_grid, moves_8n, map_copy)
    
    if path_grid is None:
        print("no path found")
        return grid, None, None, None
    simplified_path = simplify_path(path_grid)
    metric_path = transform_grid_to_metric(simplified_path, map_x, map_y, grid)

    return grid, path_grid, simplified_path, metric_path


def transform_grid_to_metric(path, map_width, map_height, grid):
    """
    Transforms a path from grid coordinates to metric (image) coordinates.

    Parameters:
    - path (List[Tuple[int, int]]): The path in grid coordinates.
    - map_width (int): Width of the map in metric (image) coordinates.
    - map_height (int): Height of the map in metric (image) coordinates.
    - grid (numpy.ndarray): The grid representing the map.

    Returns:
    - List[Tuple[float, float]]: The path in metric (image) coordinates.

    Example:
    >>> path = [(0, 0), (3, 0), (3, 2)]
    >>> map_width = 600
    >>> map_height = 400
    >>> grid = np.zeros((4, 4), dtype=int)
    >>> metric_path = transform_grid_to_metric(path, map_width, map_height, grid)
    """
    metric_path = []
    grid_x = grid.shape[1]
    grid_y = grid.shape[0]
    for x,y in path:
        metric_path.append(((x * map_width) / grid_x, (y * map_height) / grid_y))

    return metric_path


if __name__ == "__main__":
    pass

### Bayesian Filter

##### Start the client and wait for a lock

In [ ]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

### Local navigation